In [1]:
from amplpy import AMPL, ampl_notebook
ampl = ampl_notebook(
    modules=["cplex"],
    license_uuid="2563917c-d437-4bbb-afcb-592f3a4abcea")

Licensed to AMPL Community Edition License for <cs113invop@gmail.com>.


In [2]:
%%ampl_eval
reset;
set ITEMS;

param weight{ITEMS};
param value{ITEMS};
param max_weight;

var x{ITEMS} binary;

maximize total_value: sum{i in ITEMS} value[i] * x[i];

subject to weight_constraint: sum{i in ITEMS} weight[i] * x[i] <= max_weight;

In [3]:
from random import randint

items = list(range(1, 100+1))
ampl.set["ITEMS"] = items
ampl.param["weight"] = {i: randint(10, 20) for i in items}
ampl.param["value"] = {i: randint(100, 200) for i in items}
ampl.param["max_weight"] = 100
ampl.option["solver"] = "cplex"
ampl.solve()
df = ampl.get_data('x').to_pandas()

df[(df.x == 1)]


CPLEX 22.1.1.0: optimal integer solution; objective 1641
4 MIP simplex iterations
0 branch-and-bound nodes


,x
15,1
28,1
30,1
57,1
75,1
89,1
91,1
92,1
97,1


A partir de este segmento el codigo está sobre pyomo, es necesario descargar el solver en la maquina virtual.

In [4]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

In [5]:
model = pyo.ConcreteModel()

model.x = pyo.Var([1,2], domain=pyo.NonNegativeReals)
model.OBJ = pyo.Objective(expr = 2*model.x[1] + 3*model.x[2],sense=pyo.maximize)
model.Constraint1 = pyo.Constraint(expr = 3*model.x[1] + 4*model.x[2] >= 1)

In [12]:
solver = SolverFactory('glpk')
results = solver.solve(model, tee=True)

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /tmp/tmpqr8lgx_t.glpk.raw --wglp /tmp/tmp8mlwsdua.glpk.glp --cpxlp
 /tmp/tmpkomsjzmu.pyomo.lp
Reading problem data from '/tmp/tmpkomsjzmu.pyomo.lp'...
1 row, 2 columns, 2 non-zeros
18 lines were read
Writing problem data to '/tmp/tmp8mlwsdua.glpk.glp'...
11 lines were written
GLPK Simplex Optimizer 5.0
1 row, 2 columns, 2 non-zeros
Preprocessing...
1 row, 2 columns, 2 non-zeros
Scaling...
 A: min|aij| =  3.000e+00  max|aij| =  4.000e+00  ratio =  1.333e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 1
      0: obj =  -0.000000000e+00 inf =   1.000e+00 (1)
      1: obj =   7.500000000e-01 inf =   0.000e+00 (0)
LP HAS UNBOUNDED PRIMAL SOLUTION
glp_simplex: unable to recover undefined or non-optimal solution
If you need actual output for non-optimal solution, use --nopresol
Time used:   0.0 secs
Memory used: 0.0 Mb (39693 bytes)
Writing basic solution to '/t

V.D:
    x[1], x[2]

F.O:
    max  2x[1] + 3x[2]

s.a:
    3x[1] + 4x[2] >= 1